In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [43]:
base_documents_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_base_documents.parquet"
)
document_id2source = dict(zip(base_documents_df["id"], base_documents_df["source"]))

In [44]:
base_texts_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_base_text_units.parquet"
)
chunk_id_to_document_ids = dict(zip(base_texts_df["id"], base_texts_df["document_ids"]))

In [48]:
chunk_id = "23afb11229f218353d4be66e9a255f5c"
document_id2source[chunk_id_to_document_ids[chunk_id][0]]

'3b0ffec5-b1b9-4f7f-b77f-6be9224a75eb_1'

In [51]:
text_df[text_df.id=="23afb11229f218353d4be66e9a255f5c"].values[0][1]

'However, tensions have been rising between Zynara Prime and the smaller but fiercely\nindependent planet Drakalon, ruled by Governor Iska Torvath.'

In [28]:
final_community_reports_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
)

final_community_reports_df.head()

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# The Alliance Community\n\nThe Alliance commu...,1,8.0,The Alliance Community,The severity rating is high due to the entity'...,The Alliance community consists of member coun...,[{'explanation': 'This is supported by data fr...,"{\n ""title"": ""The Alliance Community"",\n ...",0e5f0f79-7f1f-4331-b1d4-e59380282ad2
1,11,# Zynara Community\n\nThe Zynara community is ...,1,6.0,Zynara Community,The community has a moderate impact severity r...,The Zynara community is centered around Chance...,"[{'explanation': 'As Chancellor of Zynara, Sel...","{\n ""title"": ""Zynara Community"",\n ""summ...",7788c5b1-0515-4159-a47f-6d28ee28c392
2,12,# Zynara Prime Alliance Community\n\nThe Zynar...,1,8.0,Zynara Prime Alliance Community,High impact due to the significant tensions be...,The Zynara Prime Alliance community consists o...,[{'explanation': 'The relationship between Dra...,"{\n ""title"": ""Zynara Prime Alliance Communi...",d6fa1abe-6af9-40c2-969b-b2ed60755046
3,13,# COALITION Community\n\nThe COALITION communi...,1,8.0,COALITION Community,The COALITION community has a moderate impact ...,The COALITION community is a collective union ...,[{'explanation': 'According to data [Data: Ent...,"{\n ""title"": ""COALITION Community"",\n ""s...",04fa7c15-6609-4553-8f4b-1cc0cd85e022
4,6,# Agentic-Systems Community\n\nThe Agentic-Sys...,1,8.0,Agentic-Systems Community,The community's reliance on technology and nee...,The Agentic-Systems community consists of a re...,[{'explanation': 'This is supported by data [D...,"{\n ""title"": ""Agentic-Systems Community"",\n...",c2eedc74-745e-474f-aa0d-9db47db4140b


In [47]:
final_community_reports_df.community.values

array(['10', '11', '12', '13', '6', '7', '8', '9', '0', '1', '2', '3',
       '4', '5'], dtype=object)

In [32]:
final_community_reports_df.keys()

Index(['community', 'full_content', 'level', 'rank', 'title',
       'rank_explanation', 'summary', 'findings', 'full_content_json', 'id'],
      dtype='object')

In [31]:
for key, value in final_community_reports_df.items():
    print("-----------------------------------")
    print(key)
    print(value.values[0])

# Neo4j Import of GraphRAG Result Parquet files

-----------------------------------
community
10
-----------------------------------
full_content
# The Alliance Community

The Alliance community consists of member countries Kalinda and Zynara, with Prime serving as Chairperson. However, the entity conducting unauthorized mining operations has undermined the Alliance's authority.

## The entity conducting unauthorized mining operations is undermining the Alliance

This is supported by data from Relationships (54) and Entities (58). The entity, led by Mark Evans, has been conducting unauthorized mining operations in the Nebular Zone [Data: Relationships (52); Entities (88)]. This has undermined the Alliance's authority and stability. The entity's actions have also affected the Unified Trade Accord, which would impact the Alliance and its member countries [Data: Relationships (48)].

## Prime serves as Chairperson of the Alliance

This is supported by data from Relationships (49). Prime works with Jenny to maintain peace among systems 

# Neo4j Import of GraphRAG Result Parquet files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. 

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](https://dev.assets.neo4j.com/wp-content/uploads/graphrag-neo4j-visualization.png)

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [22]:
GRAPHRAG_FOLDER = "/Users/luwi/Documents/Code/microsoft_graphrag_local/ragdirs/48c81e73-c15e-40c7-980a-09bc4846769f/output"

### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [3]:
%pip install --quiet pandas neo4j-rust-ext


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [5]:
NEO4J_URI = "neo4j://localhost"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "your_password"  # your password
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [6]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [7]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [8]:
pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet")

,id,text_unit_ids,source,raw_content,timestamp
0,d24a452426a5eb13a5794f6bf14734e0,[d571dbc746fecd52b0b77d9a2ddbe2f1],336701d2-baeb-45e7-a2d1-ab7f3f7c77d3_1,Abstract\nThe use of retrieval-augmented gener...,2022-12-17
1,cbf7452c8dde808317839dd1b8cea1cd,[c9b1bfb3487602ac09f6c3d6e46ed496],336701d2-baeb-45e7-a2d1-ab7f3f7c77d3_1,enables large language models (LLMs) to answer...,2022-12-17
2,9ff2665a8730d3939531f4c34e928952,[5e6b186ad6c0797210dabd31b02eb514],336701d2-baeb-45e7-a2d1-ab7f3f7c77d3_1,"However, RAG fails on global questions directe...",2022-12-17
3,a1c0bc86a1e2bac481caa1d85faf4f13,[ef2b9372b039a598ee50f0122087b69f],336701d2-baeb-45e7-a2d1-ab7f3f7c77d3_1,since this is inherently a query-\nfocused sum...,2022-12-17
4,793804a42da37467efcceb3057f49d1f,[f51800199c32e7c85bf974c278111a9c],336701d2-baeb-45e7-a2d1-ab7f3f7c77d3_1,"Prior QFS methods,\nmeanwhile, fail to scale t...",2022-12-17
...,...,...,...,...,...
65,0c6369dd2f8ba433d721aa6c78e20ea0,[5da9fe84e484ae64b8c0dba3895aaf40],1f62d343-9221-48db-b365-83e1376494e0_1,by developing new techniques for\nhealing and ...,2022-12-17
66,0e38c493febf637a633f32cef5a7431b,[c90c64681134f47580408873e5d6cbe0],1f62d343-9221-48db-b365-83e1376494e0_1,This\ninnovation has sparked fierce competitio...,2022-12-17
67,c40b95fdd2562cab858894cc1f6733e2,[e632f91242452dbb9076eaaaef8fd25f],1f62d343-9221-48db-b365-83e1376494e0_1,"Despite their rivalry, both guilds face a comm...",2022-12-17
68,7ce9d1e2760200a32d7ce84e95c16337,[35a6571f619a11c56998a3a519ce740e],1f62d343-9221-48db-b365-83e1376494e0_1,a shadowy organization that seeks to control a...,2022-12-17


In [9]:
doc_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet", columns=["id", "raw_content"]
)
doc_df.head(2)

,id,raw_content
0,d24a452426a5eb13a5794f6bf14734e0,Abstract\nThe use of retrieval-augmented gener...
1,cbf7452c8dde808317839dd1b8cea1cd,enables large language models (LLMs) to answer...


In [10]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'properties_set': 70}
70 rows in 5.115211009979248 s.


70

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [11]:
text_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_text_units.parquet",
    columns=["id", "text", "n_tokens", "document_ids"],
)
text_df.head(2)

,id,text,n_tokens,document_ids
0,23afb11229f218353d4be66e9a255f5c,"However, tensions have been rising between Zyn...",33,[018cae205b1614f5217cc20006086b3c]
1,acf18ed79e09fd944d8f9ca0658f5dcc,"HelixTech, founded by visionary inventor\nKael...",35,[02ca3ba3024a9be23c221f28ec2171fe]


In [12]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'properties_set': 140}
70 rows in 2.826556921005249 s.


70

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

In [13]:
entity_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_entities.parquet",
    columns=[
        "name",
        "type",
        "description",
        "human_readable_id",
        "id",
        "description_embedding",
        "text_unit_ids",
    ],
)
entity_df.head(2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,DRAKALON,GEO,Here is a comprehensive summary of the data pr...,0,fa417853b7004df5a959a15080815455,"[0.006349575222935646, 0.07673016767601923, -0...","[23afb11229f218353d4be66e9a255f5c, 825747b9333..."
1,GOVERNOR ISKA TORVATH,PERSON,"Governor of Drakalon, which has rising tension...",1,3435fc738c324b9bbdf2dca4c3c9b52f,"[-0.004644934247939479, 0.07013827527304689, -...",[23afb11229f218353d4be66e9a255f5c]


In [14]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 285, 'relationships_created': 343, 'nodes_created': 285, 'properties_set': 1140}
285 rows in 11.774296045303345 s.


285

### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [15]:
rel_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_relationships.parquet",
    columns=[
        "source",
        "target",
        "id",
        "rank",
        "weight",
        "human_readable_id",
        "description",
        "text_unit_ids",
    ],
)
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,DRAKALON,ZYNARA PRIME,b03aeb0eb24a471da2e610dba9734de3,13,8.0,0,There are rising tensions between the two plan...,[23afb11229f218353d4be66e9a255f5c]
1,DRAKALON,GOVERNOR ISKA TORVATH,92f71ca6f93e41d688015077ea70ae0c,10,4.0,1,Governor Iska Torvath is the ruler of Drakalon...,[23afb11229f218353d4be66e9a255f5c]


In [52]:
readable_id_2_text_unit_ids = dict(zip(rel_df["human_readable_id"], rel_df["text_unit_ids"]))


In [58]:
document_id2source[chunk_id_to_document_ids[readable_id_2_text_unit_ids["0"][0]][0]]

'3b0ffec5-b1b9-4f7f-b77f-6be9224a75eb_1'

In [16]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 252, 'properties_set': 1512}
252 rows in 4.2740678787231445 s.


252

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [17]:
community_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_communities.parquet",
    columns=["id", "level", "title", "text_unit_ids", "relationship_ids"],
)

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,4,0,Community 4,"[23afb11229f218353d4be66e9a255f5c,825747b9333a...","[b03aeb0eb24a471da2e610dba9734de3, 92f71ca6f93..."
1,3,0,Community 3,"[46933b653da989ef0265dfc323f92c3d, 1714736066d...","[b6b2319d1c63481ca3a949d87528c23e, 91118a9d7c3..."


In [18]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 14, 'relationships_created': 151, 'nodes_created': 14, 'properties_set': 42}
14 rows in 11.583256006240845 s.


14

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [19]:
community_report_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
    columns=[
        "id",
        "community",
        "level",
        "title",
        "summary",
        "findings",
        "rank",
        "rank_explanation",
        "full_content",
    ],
)
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,0e5f0f79-7f1f-4331-b1d4-e59380282ad2,10,1,The Alliance Community,The Alliance community consists of member coun...,[{'explanation': 'This is supported by data fr...,8.0,The severity rating is high due to the entity'...,# The Alliance Community\n\nThe Alliance commu...
1,7788c5b1-0515-4159-a47f-6d28ee28c392,11,1,Zynara Community,The Zynara community is centered around Chance...,"[{'explanation': 'As Chancellor of Zynara, Sel...",6.0,The community has a moderate impact severity r...,# Zynara Community\n\nThe Zynara community is ...


In [20]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 99, 'relationships_created': 99, 'nodes_created': 99, 'properties_set': 381}
14 rows in 6.831121921539307 s.


14

### Importing Covariates

Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

In [21]:
cov_df = (pd.read_parquet(f"{GRAPHRAG_FOLDER}/create_final_covariates.parquet"),)
#                         columns=["id","text_unit_id"])
cov_df.head(2)
# Subject id do not match entity ids

FileNotFoundError: [Errno 2] No such file or directory: '/Users/luwi/Documents/Code/microsoft_graphrag_local/ragdirs/48c81e73-c15e-40c7-980a-09bc4846769f/output/create_final_covariates.parquet'

In [ ]:
# Import covariates
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

{'_contains_updates': True, 'labels_added': 89, 'relationships_created': 89, 'nodes_created': 89, 'properties_set': 1061}
89 rows in 0.13370895385742188 s.


89

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTIY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag